This is a cleaned-up version of my data exploration notebook. Things were starting to get cluttered in the other one.

### Potential Inquiries
* See where properties with multiple transactions gained and lost value --> (How many houses, how much value, by town, over different periods of time)
* Foreclosures-- where and when have there been a lot?
* Building of new developments? (Might be shown by selling many houses in a short period of time on a new road)
* Signs of house flipping, i.e. a purchase and a sale for significantly more within a short period
* Has recovery been different for different segments of the market (different price-range houses)?
* Are there observable effects of the crumbling fundations in the northeastern part of CT?

### Calculations
* Adjust sales prices for inflation/season -- there is a Python package for seasonal adjustment
* Take a close look at the assessment column
* Each town's assessment rate, or look at a hosue's sale ratio relative to its town only
* Distribution of house prices in given towns, or on given streets
* Town-by-town medians, ranges

In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### Update MAX_YEAR when new data comes out.

In [2]:
MIN_YEAR = 2001
MAX_YEAR = 2016 # update this when new data comes out

#### Load in the clean data

In [3]:
def pull_out_recorded_year_and_month(df):
    """
    Returns a copy of df with columns Month and Year, which are extracted from DateRecorded.
    """
    new_df = df.copy()
    new_df['Year'] = new_df['DateRecorded'].str[6:10]
    new_df['Month'] = new_df['DateRecorded'].str[0:2]
    
    #new_df['Year'] = new_df['Year'].astype(int)
    
    return new_df

In [4]:
def combine_data_into_master_df():
    """
    Returns a DataFrame which combines all of the clean CSVs for each year.
    """
    # Combine the year-by-year clean csvs, which are located at 'data/clean_data_20xx_listings.csv'
    df = pd.read_csv('data/clean_data_' + str(MIN_YEAR) + '_listings.csv')
    for year in range(MIN_YEAR+1, MAX_YEAR+1):
        df = df.append(pd.read_csv('data/clean_data_' + str(year) + '_listings.csv'))

    # Now remove the index column
    #df = df.drop('Unnamed: 0', 1)
    return pull_out_recorded_year_and_month(df) # Pull out month, year of DateRecorded and return

In [5]:
df = combine_data_into_master_df()

In [ ]:
df.head()

### Notes on the data frames
* Use DataFrame 'df' if you want to look at ALL of the clean data
* Use 'use_df' if you want to just look at the rows that don't have NonUseCodes. You should use use_df if you are calculating any statistics.
* Use 'res_df' if you want to look at RESIDENTIAL properties (but not condos or apartments) that don't have NonUseCodes. Best way to standardize.

In [6]:
def get_residential(df):
    """
    Returns just the residential properties.
    """
    return df[df['PropertyType'] == 'Residential']

In [7]:
def get_commercial(df):
    """
    Returns just the commercial properties.
    """
    return df[df['PropertyType'] == 'Commercial']

In [8]:
use_df = df[df['NonUseCode'] < 1]
res_df = get_residential(use_df)

In [ ]:
res_df.head()

### Calculate town-by-town statistics

#### Dataframes:
* town_df for all residential properties without NonUseCodes in time range MINYEAR - MAXYEAR

#### Medians

In [9]:
# Make a dataframe with each town. We will store summary statistics in this dataframe.
town_df = pd.DataFrame(columns=['Town', 'MedianAssessedValue', 'MedianSaleAmount', 'MedianSalesRatio',
                               'MinAssessedValue', 'MaxAssessedValue', 'MinSaleAmount', 'MaxSaleAmount',
                               'MinSalesRatio', 'MaxSalesRatio'])
town_df['Town'] = df['Town'].unique()

In [10]:
def calculate_median_for_town(source_df, town, column, year, residential=True):
    """
    Calculates the median value of 'column' for a given town from df.
    NOTE: This is the median assessed value for SOLD properties. Will be
    different from the median assessed value for ALL properties.
    
    source_df should omit properties with a NonUseCode.
    """
    # Grab just the town and year of interest
    just_this_town = source_df[(source_df['Town'] == town) & (source_df['Year'] == str(year))]
    
    if residential==True:
        just_this_town = get_residential(just_this_town)
    
    return just_this_town[column].median()

In [16]:
def calculate_medians_for_all_towns(town_df_arg, source_df, column, year=MAX_YEAR, residential=True):
    """
    Calculates the median value of 'column' for ALL unique towns in df.
    Returns this information as a DataFrame
    """
    median_column_name = 'Median' + column
    new_town_df = town_df_arg.copy()
    
    # Iterate through each town and calculate its median.
    for index in new_town_df.index:
        town = new_town_df.loc[index, 'Town']
        new_town_df.loc[index, median_column_name] = calculate_median_for_town(source_df, town, column, str(year), residential)
    
    return new_town_df

In [12]:
# Now update town_df to hold the medians
town_df = calculate_medians_for_all_towns(town_df, res_df, 'AssessedValue', 2016)
town_df = calculate_medians_for_all_towns(town_df, res_df, 'SaleAmount', 2016)
town_df = calculate_medians_for_all_towns(town_df, res_df, 'SalesRatio', 2016)

In [15]:
%save town_df

'' was not found in history, as a file, url, nor in the user namespace.


#### Mins and maxes

In [17]:
def get_town_min(source_df, town_df, town, column, year=MAX_YEAR):
    """
    Finds the row with the minimum value of column for a given town in source_df.
    Saves this to town_df (as a separate copy).
    
    source_df should be residential properties w/o NonUseCodes if you are using res_df.
    """
    # Get data for just the specified town and year
    just_this_town = source_df[(source_df['Town'] == town) & (source_df['Year'] == str(year))]
    
    min_val = just_this_town[column] # a list of values to find the minimum from
    min_val = min_val.min() # Grab the minimum column value
    min_row = just_this_town[just_this_town[column] == min_val] # Grab the row
    return min_row

In [18]:
def get_town_max(source_df, town_df, town, column, year=MAX_YEAR):
    """
    Finds the row with the maximum value of column for a given town in source_df.
    Saves this to town_df (as a separate copy).
    
    source_df should be residential properties w/o NonUseCodes if you are using res_df.
    """
    # Get data for just the specified town and year
    just_this_town = source_df[(source_df['Town'] == town) & (source_df['Year'] == str(year))]
    
    town_df = just_this_town[column] # a list of values to find the minimum from
    max_val = town_df.max() # Grab the minimum column value
    max_row = just_this_town[just_this_town[column] == max_val] # Grab the row
    return max_row

In [20]:
def calculate_town_mins_and_maxes(source_df, town_df, column, year=MAX_YEAR):
    """
    Will use the data from source_df to calculate mins and maxes for column for every town.
    """
    new_town_df = town_df.copy()
    
    # Calculate min and max for each town
    for index in town_df.index:
        max_val = get_town_max(source_df, town_df, town_df.loc[index, 'Town'], column, year)[column].iloc[0]
        min_val = get_town_min(source_df, town_df, town_df.loc[index, 'Town'], column, year)[column].iloc[0]
    
        # Add to new_town_df
        min_column_name = 'Min' + column
        max_column_name = 'Max' + column
        new_town_df.loc[index, min_column_name] = min_val
        new_town_df.loc[index, max_column_name] = max_val
    
    return new_town_df

In [25]:
town_df = calculate_town_mins_and_maxes(res_df, town_df, 'SaleAmount')
town_df = calculate_town_mins_and_maxes(res_df, town_df, 'SalesRatio')
town_df = calculate_town_mins_and_maxes(res_df, town_df, 'AssessedValue')

In [26]:
%store town_df

Stored 'town_df' (DataFrame)


In [27]:
town_df.head()

,Town,MedianAssessedValue,MedianSaleAmount,MedianSalesRatio,MinAssessedValue,MaxAssessedValue,MinSaleAmount,MaxSaleAmount,MinSalesRatio,MaxSalesRatio
0,Ansonia,122600.000,199000.000,0.643,61600,234800,64900.000,307000.000,0.425,1.132
1,Ashford,137300.000,196000.000,0.684,66900,263600,84900.000,325000.000,0.473,0.992
2,Avon,250155.000,362500.000,0.713,62220,1908500,72000.000,2500000.000,0.500,1.032
3,Barkhamsted,168860.000,257750.000,0.678,119300,386460,168000.000,560000.000,0.567,0.834
4,Berlin,166300.000,260000.000,0.662,65000,415900,88000.000,592000.000,0.467,0.972


### Timeseries functionality
Should be able to make timeseries for individual towns and all towns on:
* Median of any column
* Number sold in each year

#### Median timeseries

In [ ]:
def get_years_median(column, year, town='Connecticut', source_df=res_df):
    """
    Returns the median value of column for town from year, using data from source_df, as a list.
    This is a HELPER FUNCTION for median_timeseries()
    """
    
    if town != 'Connecticut':
        # Get the rows for this town in the given year
        just_this_town = source_df[source_df['Town'] == town]
    else:
        # Look at all the data from the state
        just_this_town = source_df

    # Grab just the year you want
    town_by_year = just_this_town[just_this_town['Year'] == year]

    # Calculate and return the median
    median = town_by_year[column].median()
    if median == 'nan':
        return -1
    else:
        return median

In [ ]:
get_years_median('SaleAmount', 2016)

In [ ]:
def median_timeseries(column, town='Connecticut', startyear=MIN_YEAR, endyear=MAX_YEAR, source_df=res_df):
    """
    Returns a time series for a particular town and column, from start year to endyear.
    Values are the median of column.
    Grabs the data from source_df.
    """
    # Make the DataFrame's attributes
    index = list(range(startyear, endyear+1))
    columns = [town]
    
    # Generate a list of medians using helper function
    list_of_medians = []
    for year in index:
        list_of_medians.append(get_years_median(column, year, town))
        
    return pd.DataFrame(data=list_of_medians, index=index, columns=columns)

In [ ]:
def median_timeseries_all_towns(column, startyear=MIN_YEAR, endyear=MAX_YEAR, source_df=res_df):
    """
    Returns a time series of all towns in CT, as well as the CT average.
    Each column is a year with values being the median of 'column' for that town in that year.
    """
    # Make the DataFrame's attributes
    columns = list(range(startyear, endyear+1))
    time_series = pd.DataFrame(columns=columns, data = median_timeseries(column).transpose())
    
    # Add in the info for each town
    for town in source_df['Town'].unique():
        time_series = pd.concat([time_series, median_timeseries(column, town).transpose()])
        
    # Add in town name column
    time_series['Town'] = time_series.index
            
    return time_series

In [ ]:
# Save the saleamount timeseries for all towns for use later.
saleamount_ts_all = median_timeseries_all_towns('SaleAmount')
%save saleamount_ts_all

In [ ]:
saleamount_ts_all.head()

#### Number sold time series

In [ ]:
def get_years_sale_count(year, town='Connecticut', source_df=res_df):
    """
    Returns the nummber of sales in one town from year, using data from source_df, as a list.
    This is a HELPER FUNCTION for sale_count_timeseries().
    """
    
    if town != 'Connecticut':
        # Get the rows for this town in the given year
        just_this_town = source_df[source_df['Town'] == town]
    else:
        # Look at all the data from the state
        just_this_town = source_df

    # Grab just the year you want
    town_by_year = just_this_town[just_this_town['Year'] == year]

    # Calculate and return the median
    count = len(town_by_year)
    if count == 'nan':
        return -1
    else:
        return count

In [ ]:
def sale_count_timeseries(town='Connecticut', startyear=MIN_YEAR, endyear=MAX_YEAR, source_df=res_df):
    """
    Returns a time series for a particular town and column, from start year to endyear.
    Values are the number of sales in that town in that year.
    Grabs the data from source_df.
    """
    # Make the DataFrame's attributes
    index = list(range(startyear, endyear+1))
    columns = [town]
    
    # Generate a list of medians using helper function
    list_of_sale_counts = []
    for year in index:
        list_of_sale_counts.append(get_years_sale_count(year, town))
        
    return pd.DataFrame(data=list_of_sale_counts, index=index, columns=columns)

In [ ]:
def sale_count_timeseries_all_towns(startyear=MIN_YEAR, endyear=MAX_YEAR, source_df=res_df):
    """
    Returns a time series of all towns in CT, as well as the CT average.
    Each column is a year with values being the number of homes sold in that town in that year.
    """
    # Make the DataFrame's attributes
    columns = list(range(startyear, endyear+1))
    time_series = pd.DataFrame(columns=columns, data = sale_count_timeseries('Connecticut', startyear, endyear).transpose())
    
    for town in source_df['Town'].unique():
        time_series = pd.concat([time_series, sale_count_timeseries(town, startyear, endyear).transpose()])
            
    return time_series

In [ ]:
sale_count_ts_all_towns = sale_count_timeseries_all_towns()
%store sale_count_ts_all_towns

### Graphing the number of homes sold over time with the median SaleAmount over time

In [ ]:
# Increase number sold by a factor of x so the line on the plot doesn't look flat (to even out their scales)
def scale_df_row(df, row_index, factor):
    """
    Returns df where the row at row_index is scaled by factor.
    To be used to scale data up or down so that they can be plotted on top of other data.
    This is a HELPER FUNCTION for combined_sales_and_median_sale_amount_timeseries().
    """
    new_df = df.copy()
    new_df.iloc[row_index] = new_df.iloc[row_index].apply(lambda x: x*factor)
    return new_df

In [ ]:
def combine_sales_and_median_sale_amount(town='Connecticut', startyear=MIN_YEAR, endyear=MAX_YEAR, source_df = res_df):
    """
    Returns a dataframe where the first row is the MedianSaleAmount timeseries of town,
    and the second row is the number of homes sold in that town.
    This function feeds ___.
    """
    combined_df = median_timeseries('SaleAmount', town).transpose().append(sale_count_timeseries(town).transpose())
    combined_df.index = ['MedianSaleAmount', 'NumberSold']
    return combined_df

In [ ]:
def combined_sales_and_median_sale_amount_timeseries(town='Connecticut', scale_factor=500, startyear=MIN_YEAR,
                                                     endyear=MAX_YEAR, source_df=res_df):
    """
    Returns a timeseries ready to be plotted that graphs the number of sales AND the median sale amount in
    a town, every year from startyear to endyear.
    
    The argument 'scale_factor' is a number to scale the number of sales row by, in order for this line to not appear flat
    beside median sale amount when graphed.
    """
    timeseries = combine_sales_and_median_sale_amount(town, startyear, endyear, source_df)
    timeseries = scale_df_row(timeseries, 1, scale_factor)
    return timeseries.transpose()

### Fact check your SaleAmount medians and NumberSold

#### Cross-check median sale amount with Warren Group:
"Meanwhile, the median price of a single-family home increased 1.9 percent in August to \$265,000, compared with __\$260,000__ a year ago." from https://thewarrengroup.com/2017/09/26/conn-single-family-edge-higher-in-august/

In [ ]:
saleamount_ts_all[saleamount_ts_all['Town'] == 'Connecticut'][2016]

#### Cross-check median sale price and  number sold with CTRealtors:
"Connecticut REALTORS® reports that the single-family residential home median sales price is $237,000 which
reflects a 3.1 percent increase from $229,900 in that same time period last year. Median indicates that half the
homes sold for more and half for less. Single-family residential home sales in Connecticut increased 5.1 percent
comparing March 2017 to March 2016. The total units of homes sold were 2,527 in March 2017 and 2,404 in
March 2016." from https://www.ctrealtors.com/Customer-Content/www/press-releases/PDFs/Stats-2017March.pdf

In [ ]:
mar_2016 = res_df[(res_df['Year'] == '2016') & (res_df['Month'] == '03')]


#### Cross-check numer sold with CTRealtors:
"The total units of homes sold were 2,307 in January 2017 and 2,020 in January 2016." (From https://ctrealtors.com/Customer-Content/www/press-releases/PDFs/Stats-2017January.pdf)

(*Note:* I am unable to find info on the *total* number of sales in 2016, but WG and CTRealtors do mention monthly sale numbers in their reports.)

In [ ]:
jan_2016 = res_df[(res_df['Year'] == '2016') & (res_df['Month'] == '01')]
print(len(jan_2016)) # Should be close to 2,020

#### Try with WG August report
"A total of 3,829 single-family homes were sold in Connecticut during the month of August, compared with 3,776 sold in August 2016." from https://thewarrengroup.com/2017/09/26/conn-single-family-edge-higher-in-august/

In [ ]:
aug_2016 = res_df[(res_df['Year'] == '2016') & (res_df['Month'] == '09')]
print(len(aug_2016)) # Should be close to 3,776

### Load in population and median income data for each town to help with town subsetting.
* Town-by-town population estimates come from CT DPH: https://portal.ct.gov/DPH/Health-Information-Systems--Reporting/Population/Annual-Town-and-County-Population-for-Connecticut
* Median income data from Census Bureau via CT Data Collaborative http://data.ctdata.org/dataset/median-household-income-by-town

#### Population data

In [ ]:
# Clean town population data by separating the pairs of columns and concatenate them into one datafame
def clean_town_population_df(raw_df):
    """
    Returns a clean version of the raw town population df.
    """
    clean_df = raw_df.copy()
    
    # Split up column pairs
    town_pop_section_1 = clean_df[['Town', 'Est. Pop.']]
    town_pop_section_2 = clean_df[['Town.1', 'Est. Pop..1']]
    town_pop_section_3 = clean_df[['Town.2', 'Est. Pop..2']]
    town_pop_section_4 = clean_df[['Town.3', 'Est. Pop..3']]
    
    # Trim off the NaN rows at the end of each section
    town_pop_section_1 = town_pop_section_1[0:-9]
    town_pop_section_2 = town_pop_section_2[0:-9]
    town_pop_section_3 = town_pop_section_3[0:-9]
    town_pop_section_4 = town_pop_section_4[0:-8]
    
    # Fix column names in order to concatenate properly
    columns = ['Town', 'Population']
    town_pop_section_1.columns = columns
    town_pop_section_2.columns = columns
    town_pop_section_3.columns = columns
    town_pop_section_4.columns = columns
    
    clean_df = pd.concat([town_pop_section_1, town_pop_section_2, town_pop_section_3, town_pop_section_4]).reset_index(drop=True)


    for index in clean_df.index:
        # remove commas
        clean_df.loc[index, 'Population'] = str(clean_df.iloc[index]['Population']).replace(',','')
    
    clean_df['Population'] = clean_df['Population'].astype(float)
    
    return clean_df

In [ ]:
town_pop_df = clean_town_population_df(
pd.read_excel("https://portal.ct.gov/-/media/Departments-and-Agencies/DPH/Population/Town-Pop/pop_towns2016xlsx.xlsx?la=en",
                             skiprows=10))

In [ ]:
def clean_town_income_df(raw_df):
    """
    Returns a clean version of raw town income data.
    Grabs info for each town where Race/Ethnicity = All, Measure Type = Number, Variable = Median Household Income,
    and Year = 2012-2016.
    """
    clean_df = raw_df.copy()
    
    # Grab only select info
    clean_df = clean_df[(clean_df['Race/Ethnicity'] == 'All') & (clean_df['Year'] == '2012-2016')
                       & (clean_df['Measure Type'] == 'Number') & (clean_df['Variable'] == 'Median Household Income')]
    
    # Put Connecticut at the top of the dataframe while maintaining alphabetical order otherwise
    ct_row = clean_df[clean_df['Town'] == 'Connecticut'] # Grab CT row
    clean_df = clean_df[clean_df['Town'] != 'Connecticut'] # Remove it from middle of df
    clean_df = pd.concat([ct_row, clean_df]) # Add CT back to top
    
    # Drop unwanted columns
    clean_df = clean_df[['Town', 'FIPS', 'Value']]
    
    # Rename columns
    clean_df.columns = ['Town', 'FIPS', 'MedianHouseholdIncome']
    
    return clean_df.reset_index(drop=True)

In [ ]:
town_income_df = clean_town_income_df(pd.read_csv("http://data.ctdata.org/dataset/68296d39-d0f7-4e2d-ba0e-98a3463ab58f/resource/1a962569-c6b3-4031-b7b1-f051e9c12379/download/median-household-income-town-2016.csv"))

#### Now merge this new info into town_df_copy

In [ ]:
def add_population_and_income_data(df, town_pop, town_income):
    """
    Assuming df has Town names, will join data from town_income_df and town_population_df by town.
    Specifically, grabs 'Population' from town_pop and 'MedianHouseholdIncome', as well as 'FIPS', from town_income.
    """
    new_df = df.copy()
    new_df = df.merge(right = town_pop[['Town','Population']], on='Town')
    new_df = new_df.merge(right = town_income[['Town','MedianHouseholdIncome','FIPS']], on = 'Town')
    return new_df

In [ ]:
town_df_copy = add_population_and_income_data(town_df, town_pop_df, town_income_df)

#### Reorder columns for better readability

In [ ]:
town_df_copy = town_df_copy[['Town', 'FIPS', 'Population', 
                   'MedianHouseholdIncome', 'MedianSaleAmount', 'MaxSaleAmount', 'MinSaleAmount',
                   'MedianAssessedValue', 'MaxAssessedValue', 'MinAssessedValue', 'MaxSalesRatio', 'MinSalesRatio']]

In [ ]:
%store town_df_copy

In [ ]:
town_df = town_df_copy

### Calculate SaleAmount recovery
Look at the peak median SaleAmount before the Recession and compare it to median SaleAmount in 2016.